# 1. Imports

In [1]:
!rm -rf test_models
!git clone https://github.com/KshRaven/test_models.git

Cloning into 'test_models'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 66 (delta 15), reused 62 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 30.15 KiB | 10.05 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [2]:
import sys
import torch

In [3]:
%cd test_models
!pip install -e .
%cd ..

/content/test_models
Obtaining file:///content/test_models
  Preparing metadata (setup.py) ... done
  Running setup.py develop for test_models
/content


In [4]:
sys.path.append('./test_models/')

In [5]:
from test_models.build import Reformer

import torch
import torch.nn as nn

# 2. Parameters

In [6]:
FEEDBACK = True
INPUT_DIMS = 11
OUTPUT_DIMS = 4
FEEDBACK_DIMS = INPUT_DIMS+ (OUTPUT_DIMS+1 if FEEDBACK else 0)
EMBED_SIZE = 32
MAX_SEQ_LEN = 512
LAYERS = 4
HEADS = 1
KV_HEADS = None
DIFFERENTIAL = True
BIAS = False
PRI_ACTV = nn.SiLU()
SEC_ACTV = nn.Sigmoid()

In [7]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DTYPE = torch.float32
DEVICE

'cpu'

# 3. Create model

In [8]:
MODEL = Reformer(INPUT_DIMS+FEEDBACK_DIMS, OUTPUT_DIMS, 1, EMBED_SIZE, MAX_SEQ_LEN, LAYERS, HEADS, KV_HEADS,
                 DIFFERENTIAL, 0.1, BIAS, FEEDBACK, DEVICE, DTYPE,
                 pri_actv=PRI_ACTV, sec_actv=SEC_ACTV, prob=False, dist='mult_var_normal')

In [9]:
MODEL.single_mode(False)

# 4. Create data

In [10]:
SEQ_LEN = 64
RECORDS = 10

In [11]:
train_inp = torch.randn(RECORDS, SEQ_LEN, INPUT_DIMS+FEEDBACK_DIMS).to(DEVICE, DTYPE)

# Test

In [13]:
torch.cuda.empty_cache()

In [14]:
with torch.no_grad():
    test_policy, test_prob = MODEL.get_action(train_inp)
MODEL.corr, MODEL.corr_indices

(Linear(in_features=8, out_features=6, bias=False),
 tensor([[1, 2, 2, 3, 3, 3],
         [0, 0, 1, 0, 1, 2]]))

In [15]:
test_policy[0, -4:].cpu().numpy()

array([[-0.1758607 ,  0.4344102 , -0.00523913,  0.6154233 ],
       [ 0.0566698 ,  0.98794186,  0.51657665,  1.2415535 ],
       [ 1.4502698 ,  0.6363247 , -1.2129441 ,  0.9213729 ],
       [ 0.1315606 , -1.0511289 ,  0.7051847 , -0.0696131 ]],
      dtype=float32)

In [16]:
test_prob[0, -4:].cpu().numpy()

array([-2.9974756, -3.2313366, -6.095659 , -5.432405 ], dtype=float32)

In [17]:
with torch.no_grad():
    test_latent = MODEL.get_latent(MODEL.pol_proj, train_inp, single=True)

In [18]:
with torch.no_grad():
    test_mean = MODEL.get_mean(test_latent)

In [19]:
test_mean[0].cpu().numpy()

array([[0.49412355, 0.50564617, 0.45550522, 0.48925295]], dtype=float32)

In [20]:
with torch.no_grad():
    test_std = MODEL.get_std(test_latent)

In [21]:
test_std[0].cpu().numpy()

array([[0.70074624, 0.79811573, 0.6881542 , 0.72398823]], dtype=float32)

In [22]:
MODEL.mean_std

Linear(in_features=32, out_features=8, bias=False)

In [23]:
test_latent.shape

torch.Size([10, 1, 32])

In [24]:
MODEL.corr, MODEL.corr_indices

(Linear(in_features=8, out_features=6, bias=False),
 tensor([[1, 2, 2, 3, 3, 3],
         [0, 0, 1, 0, 1, 2]]))

In [25]:
del MODEL.corr

In [26]:
with torch.no_grad():
    test_dist = MODEL.dist(test_mean, test_std, test_latent)

In [27]:
MODEL.corr, MODEL.corr_indices

(Linear(in_features=32, out_features=6, bias=False),
 tensor([[1, 2, 2, 3, 3, 3],
         [0, 0, 1, 0, 1, 2]]))